In [1]:
import numpy as np
import torch
import torchvision
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable

batch_size_train = 100
batch_size_test = 100

class mlp(nn.Module):
    def __init__(self):
        super(mlp, self).__init__()
        self.fc0 = nn.Linear(784,1024)
        self.fc1 = nn.Linear(1024, 10)

    def forward(self, x):
        x = x.view((-1,784))
        x = F.relu(self.fc0(x))
        return self.fc1(x)

model = mlp().cuda()
optimizer = optim.Adam(model.parameters(), lr=5e-4,)

In [2]:
def update_param(parameter_data, activation, gamma=0.1,mean = 0.1):
    activation = activation.data.numpy().mean(axis=(0)).squeeze()[:,None]
    param = parameter_data.numpy()
    new_param = param+gamma*np.clip((param*param)*(mean - activation).mean(),-1,1)
    return torch.from_numpy(new_param)
def perform_update(m):
    if type(m) == nn.Linear:
        m.eval()
        im = np.random.uniform(-1, 1, (100, m.in_features))+0.1
        im_as_ten = torch.from_numpy(im).float().cuda()
        x = Variable(im_as_ten, requires_grad=True)
        x = F.relu(m(x))
        m.weight.data = update_param(m.weight.data.cpu(),activation=x.cpu()).cuda()

In [3]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('mnist', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('mnist', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [4]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(100 + 1)]
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data=data.cuda()
        target=target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target) + 1.0 * (0.1 - output.mean())**2
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append(
            (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
    model.apply(perform_update)

In [5]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        test_loss /= len(test_loader.dataset)
        test_losses.append(test_loss)
        print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
        return 100. * correct / len(test_loader.dataset)

In [6]:
test()
accs = []
for epoch in range(1, 100):
    train(epoch)
    accs.append(test())
print(max(accs))

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3196, Accuracy: 768/10000 (8%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.320545

Test set: Avg. loss: 0.1113, Accuracy: 9669/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.055593

Test set: Avg. loss: 0.0784, Accuracy: 9756/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.035090

Test set: Avg. loss: 0.0653, Accuracy: 9807/10000 (98%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.017559

Test set: Avg. loss: 0.0602, Accuracy: 9815/10000 (98%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.029249

Test set: Avg. loss: 0.0577, Accuracy: 9818/10000 (98%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.045290

Test set: Avg. loss: 0.0609, Accuracy: 9801/10000 (98%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.007181

Test set: Avg. loss: 0.0602, Accuracy: 9819/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.016152

Test set: Avg. loss: 0.0592, Accuracy: 9828/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.015310

Test set: Avg. loss: 0.0588, Accuracy: 9828/10000 (98%)

Train Epoch: 10 [